Generate EL points and simulate SiPM responses

In [1]:
from __future__ import print_function

import random
import tables
import numpy as np
import sys

NSIPM = 8

num_EL_points = 4     # max num points observed in EL in same timestep 
sipm_pitch = 10.0     # SiPM pitch in mm
sipm_edge_width = 5.0 # width of edge of dice board in mm
ze = 10.0             # distance between SiPM plane and EL gap
grid_space = 2.0      # grid spacing in mm
d_gap = 5.0           # length of EL gap
n_tbins = 2           # number of time bins collected as electron crosses EL

N = 1   # Number of photons (probably not necessary due to normalization)


nevts = [40000,40000,1,1] # num events for each num EL pts

max_xy = (NSIPM-1)*sipm_pitch + 2*sipm_edge_width # maximum x and y value (80 mm)
max_p = max_xy /grid_space                        # number of points per line (40)


sipm_res1 = np.empty((nevts[0],NSIPM**2),dtype=np.float64) # initialize sipm responses for 1 evt
sipm_res2 = np.empty((nevts[1],NSIPM**2),dtype=np.float64) # 2
sipm_res3 = np.empty((nevts[2],NSIPM**2),dtype=np.float64) # 3
sipm_res3 = np.empty((nevts[3],NSIPM**2),dtype=np.float64) # 4

Set up the SiPM positions

In [2]:
sipm_pos_x = []
for i in range(NSIPM): sipm_pos_x.extend([sipm_edge_width + i*sipm_pitch]*NSIPM)
sipm_pos_y = [sipm_edge_width + i*sipm_pitch for i in range(NSIPM)]*NSIPM

print(zip(sipm_pos_x,sipm_pos_y))


[(5.0, 5.0), (5.0, 15.0), (5.0, 25.0), (5.0, 35.0), (5.0, 45.0), (5.0, 55.0), (5.0, 65.0), (5.0, 75.0), (15.0, 5.0), (15.0, 15.0), (15.0, 25.0), (15.0, 35.0), (15.0, 45.0), (15.0, 55.0), (15.0, 65.0), (15.0, 75.0), (25.0, 5.0), (25.0, 15.0), (25.0, 25.0), (25.0, 35.0), (25.0, 45.0), (25.0, 55.0), (25.0, 65.0), (25.0, 75.0), (35.0, 5.0), (35.0, 15.0), (35.0, 25.0), (35.0, 35.0), (35.0, 45.0), (35.0, 55.0), (35.0, 65.0), (35.0, 75.0), (45.0, 5.0), (45.0, 15.0), (45.0, 25.0), (45.0, 35.0), (45.0, 45.0), (45.0, 55.0), (45.0, 65.0), (45.0, 75.0), (55.0, 5.0), (55.0, 15.0), (55.0, 25.0), (55.0, 35.0), (55.0, 45.0), (55.0, 55.0), (55.0, 65.0), (55.0, 75.0), (65.0, 5.0), (65.0, 15.0), (65.0, 25.0), (65.0, 35.0), (65.0, 45.0), (65.0, 55.0), (65.0, 65.0), (65.0, 75.0), (75.0, 5.0), (75.0, 15.0), (75.0, 25.0), (75.0, 35.0), (75.0, 45.0), (75.0, 55.0), (75.0, 65.0), (75.0, 75.0)]


Generate array of El events

In [10]:
elpt = np.array([np.random.randint(0,1600,nevts[0]), np.random.randint(0,1600,nevts[1]),np.random.randint(0,1600,nevts[2]),np.random.randint(0,1600,nevts[3])]) 

x0 = (elpt[0] % max_p)*grid_space + 1
x1 = (elpt[1] % max_p)*grid_space + 1
x2 = (elpt[2] % max_p)*grid_space + 1
x3 = (elpt[3] % max_p)*grid_space + 1

y0 = np.floor(elpt[0]/max_p)*grid_space + 1
y1 = np.floor(elpt[1]/max_p)*grid_space + 1
y2 = np.floor(elpt[2]/max_p)*grid_space + 1
y3 = np.floor(elpt[3]/max_p)*grid_space + 1

print(x0)
print(x1)

[ 39.   3.  11. ...,  41.  73.  17.]
[ 17.  11.  43. ...,  23.   9.  13.]


Define sipm response function

In [11]:
"""
sipm_param.py
author: jrenner
Defines the SiPM parameterization functions as:
N(x) = M*sum(c_n*x^n) for n = 0 to n = 9
where x is the distance of the SiPM from some central point of
light emission.
Because the response is characterized over several time bins, we
have several values for M and the coefficients.
"""
import numpy as np

# Number of time bins
n_tbins = 2

# Coefficients from S2 parameterization
M = [1.599, 1.599]
c0 = [7.72708346764e-05, 0.000116782596518]
c1 = [-1.69330613273e-07, 3.05115354927e-06]
c2 = [-1.52173658255e-06, -7.00800605142e-06]
c3 = [-2.4985972302e-07, 6.53907883449e-07]
c4 = [1.12327204397e-07, 8.95230202525e-08]
c5 = [-1.49353264606e-08, -2.27173290582e-08]
c6 = [1.04614146487e-09, 2.00740799864e-09]
c7 = [-4.19111362353e-11, -9.21915945523e-11]
c8 = [9.12129133361e-13, 2.20534216312e-12]
c9 = [-8.40089561697e-15, -2.1795164563e-14]

# Maximum radial extent of parameterization
rmax = 20.

# Return the SiPM response for the specified time bin and radial distance.
def sipm_par(tbin,r):

    # Ensure the time bin value is valid.
    if(tbin < 0 or tbin >= n_tbins):
        print("Invalid time bin in sipm_param: returning 0.0 ...")
        return 0.0

    # Calculate the response based on the parametrization.
    vpar = M[tbin]*(c0[tbin] + c1[tbin]*r + c2[tbin]*r**2 + c3[tbin]*r**3 + 
    c4[tbin]*r**4 + c5[tbin]*r**5 + c6[tbin]*r**6 + c7[tbin]*r**7 + 
    c8[tbin]*r**8 + c9[tbin]*r**9)

    # Zero the response for radii too large.
    if(hasattr(vpar, "__len__")):
        ret = np.zeros(len(vpar)); iret = 0
        for rv,pv in zip(r,vpar):
            if(rv < rmax):
                ret[iret] = pv
            iret += 1
        return ret
    else:
        if(r < rmax):
            return vpar
        return 0.0


Generate SiPM responses

In [14]:
def gen_resp(x,y,pts):
    
    posxy = zip(sipm_pos_x,sipm_pos_y)
    R = np.empty_like(sipm_res)

    for xp,yp in zip(x,y): print(xp,yp)
    # generate responses
    
    """
    for each xp,yp in zip(x,y):
        evt = 0
        
        # for each event (xi,yi) are its coordinates
        for xi,yi in zip(xp,yp):
            idx = 0 
            # for each sipm get sipm_par
            for posx,posy in posxy:
                r = np.sqrt((posx - xi)**2 + (posy - yi)**2)
                sipm_res[evt,idx] = sipm_par(0,r) 
                #print(posx,xi,posy,yi)
                R[evt,idx] = r
                #print(r)
                if n_tbins == 2: 
                    sipm_res[evt,idx] += sipm_par(1,r)
                idx += 1

            evt += 1

    #multiply by number of photons, maybe not neccessary 
    if n_tbins == 2: sipm_res = sipm_res * N / 2 #average time of 2 bins
    else: sipm_res *= N
    
    """


Put responses in a table

In [15]:
# Store "x" in a chunked array with level 5 BLOSC compression...
f = tables.open_file('resp.h', 'w')

filters = tables.Filters(complib='blosc', complevel=9)

#x
atom = tables.Atom.from_dtype(x.dtype)
xl = f.create_carray(f.root, 'x', atom, x.shape, filters=filters)
xl[:] = x

#y
atom = tables.Atom.from_dtype(y.dtype)
yl = f.create_carray(f.root, 'y', atom, y.shape, filters=filters)
yl[:] = y

#sipm
atom = tables.Atom.from_dtype(sipm_res.dtype)
sipm_resl = f.create_carray(f.root, 'sipm_resp', atom, sipm_res.shape, filters=filters)
sipm_resl[:] = sipm_res


print(f.root.x.nrows)

#print(f.root.sipm_resp[0])
#print(R[0])

f.close()





NameError: name 'x' is not defined

Done!


Below is a very small test:
*change n_tbins to 2

In [ ]:
f = tables.open_file('resp.h', 'r')
test_evt = 0
test_sipm = 20
print(f.root.sipm_resp[0][test_sipm])


To verify this is as it should be...

In [ ]:
r = np.sqrt((x[test_evt] - sipm_pos_x[test_sipm])**2 + (y[test_evt] - sipm_pos_y[test_sipm])**2)
test = ( sipm_par(0,r) + sipm_par(1,r) )*N/2


print('r: ' + str(r) + ' *if r > 20 this should sipm response should be 0')
print('sipm response: '+ str(test) + ' = ' + str(f.root.sipm_resp[0][test_sipm]) + '?')
print('x: ' + str(x[test_evt]) + ' = ' + str(f.root.x[test_evt]) + '?')
print('y: ' + str(y[test_evt]) + ' = ' + str(f.root.y[test_evt]) + '?')

In [ ]:
f.close()